In [1]:
import os, sys
import pandas as pd
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import importlib
import utils.event_extractor  # your module

importlib.reload(utils.event_extractor)
from utils.event_extractor import EventExtractor  # re-import your class if needed


filtered_reports_df = pd.read_pickle("../exports/filtered_patient_reports.pkl")
extractor = EventExtractor(event_name_model_type="biolord", attribute_model_type="llama3")

def extract_events(sentences):
    global extractor
    event_types = ["Pain", "Sleep"]
    events = extractor.extract_events(sentences=sentences, event_names=event_types, threshold=0.2)
    return events


/home/asusaiyah/projects/event_log_from_text/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

filtered_reports_df_debug = filtered_reports_df.head(1)
filtered_reports_df_debug['Events'] = filtered_reports_df_debug['Sentences'].apply(extract_events)
filtered_reports_df["Events"] = filtered_reports_df['Sentences'].apply(extract_events)


/tmp/ipykernel_123394/3897294729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reports_df_debug['Events'] = filtered_reports_df_debug['Sentences'].apply(extract_events)


In [ ]:
filtered_reports_df.to_pickle("../exports/filtered_patient_reports_with_event_log.pkl")

[{'22 yr old with type 1 iddm  since age 6. patient admitted with n/v and glu\n   345, DKA.': {'event': 'Others',
   'attributes': {}}},
 {'Tx in ew with ns , zolfran, morphine and iv insulin.': {'event': 'Others',
   'attributes': {}}},
 {'Hx\n   depression, high chol, gastritis with dka admit on [**2-27**].': {'event': 'Others',
   'attributes': {}}},
 {'': {'event': 'Pain', 'attributes': {}}},
 {' iv insulin stopped 10 am with sc glargine at 0930,  fs on iv in\n   80\ns  now 130-160, no nausea , vomited at 0800 small amt approx 20 cc.\n   tolerating fluids and jello': {'event': 'Others',
   'attributes': {}}},
 {' increased coverage and extra dose glargine and humalog at 1630.': {'event': 'Others',
   'attributes': {}}},
 {'labs drawn at a630, iv fluids off at 1500': {'event': 'Others',
   'attributes': {}}},
 {' patient requesting solid supper and no n/v see 1630 labs': {'event': 'Others',
   'attributes': {}}},
 {' cont qid fs and coverage with SSI, give antiemetics.': {'event': '